## Dilemmas:
1. Maybe paragraph corpus, and not text (because the B1 articles have more than one, while B2-C1 articles have only one)
2. With regular news articles?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pylab as plt
from IPython.display import IFrame

import fetch_dw

plt.ion()

In [3]:
pages_df = fetch_dw.build_pages_df(100 , to_filter=False)

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving all articles URLS...


KeyboardInterrupt
2018-06-20T16:47:17Z


KeyboardInterrupt: 

## URL type (`av`)

In [ ]:
url_types = pages_df["url"].apply(lambda s: s.split("/")[-1].split("-")[0])

In [ ]:
url_types.value_counts()

In [ ]:
pages_df[url_types == "av"]["artikel"].value_counts()

In [ ]:
pages_df = pages_df[url_types != "av"]

## Levels vs. Type of Article (`artikel`)

In [ ]:
leveled_artikel_count = pages_df.groupby(["artikel", "levels"]).size()
leveled_artikel_count[leveled_artikel_count>5].dropna(how="all").fillna(0).sort_values()

### TAKEN TO CORPUS

#### Top-Thema – Podcast - B1 [One Text]


In [ ]:
page = pages_df[pages_df["artikel"] == "Top-Thema – Podcast"].iloc[0]
print(page["url"])
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Nachrichten - B2 & C1 - [Multiple Texts]

In [ ]:
page = pages_df[pages_df["artikel"] == "Nachrichten"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Langsam gesprochene Nachrichten - B2 & C1 [Multiple Texts]

In [ ]:
page = pages_df[pages_df["artikel"] == "Langsam gesprochene Nachrichten"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Top-Thema – Lektionen - B1 [One Text]

In [ ]:
page = pages_df[pages_df["artikel"] == "Top-Thema – Lektionen"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

### NOT TAKEN TO CORPUS

#### Video-Thema – Lektionen - B2 & C1 [Dialogue]

In [ ]:
page = pages_df[pages_df["artikel"] == "Video-Thema – Lektionen"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Sprachbar - C1 & C2

In [ ]:
page = pages_df[pages_df["artikel"] == "Sprachbar"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Alltagsdeutsch – Podcast - C1 & C2 [No Text at all]

In [ ]:
page = pages_df[pages_df["artikel"] == "Alltagsdeutsch"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Video-Thema – Podcast ohne UT - B2 & C1

In [ ]:
page = pages_df[pages_df["artikel"] == "Video-Thema – Podcast ohne UT"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

#### Wort der Woche  - B2, B2 & C1

In [ ]:
page = pages_df[pages_df["artikel"] == "Wort der Woche"].iloc[0]
#print(page["content"])
IFrame("http://dw.com" 
       + page["url"],
       width=1000, height=500)

## Load Full Text Dateframe

In [4]:
text_df = pd.read_hdf('dw.h5', "text_df")

In [31]:
number_paragraphs = text_df["text"].apply(lambda text: len(text.split("\n")))

In [53]:
paragrpahs_rows = []
for _, text_row in text_df[number_paragraphs != 1].iterrows():
    for paragraph in text_row["text"].split("\n"):
        paragrpahs_row = text_row.copy()
        paragrpahs_row["text"] = paragraph
        paragrpahs_rows.append(paragrpahs_row)

paragraphs_df = pd.concat([text_df[number_paragraphs == 1],
           pd.DataFrame(paragrpahs_rows)])["text"].iloc[-5]

'Zu den größten Firmen zählen die Deutsche Telekom und die Deutsche Post. Übrigens ist die Regierung nicht vollständig nach Berlin gezogen: Knapp 9000 Beamte arbeiten noch immer in Bonn – und viele Politiker pendeln zwischen den beiden Städten. Außerdem darf Bonn sich noch heute "Bundesstadt" nennen – ein Titel, der in Deutschland einmalig ist.'

In [54]:
pd.DataFrame(paragrpahs_rows)

,index,url,rubrik,y,artikel,text,title
0,0,/de/von-bonn-nach-berlin/a-15209528,DEUTSCH LERNEN,0,Top-Thema – Podcast,"Nach dem Zweiten Weltkrieg wurde beschlossen, ...",Von Bonn nach Berlin
0,0,/de/von-bonn-nach-berlin/a-15209528,DEUTSCH LERNEN,0,Top-Thema – Podcast,"Dafür ist aber auch viel Geld geflossen: 1,4 M...",Von Bonn nach Berlin
0,0,/de/von-bonn-nach-berlin/a-15209528,DEUTSCH LERNEN,0,Top-Thema – Podcast,Zu den größten Firmen zählen die Deutsche Tele...,Von Bonn nach Berlin
0,0,/de/von-bonn-nach-berlin/a-15209528,DEUTSCH LERNEN,0,Top-Thema – Podcast,Ist die Stadt nun Gewinner oder Verlierer des ...,Von Bonn nach Berlin
7,0,/de/oxford-hier-studiert-die-elite/a-2347044,DEUTSCH LERNEN,0,Top-Thema – Podcast,Mit einem Abschluss an der Universität Oxford ...,Oxford: hier studiert die Elite
7,0,/de/oxford-hier-studiert-die-elite/a-2347044,DEUTSCH LERNEN,0,Top-Thema – Podcast,Die Struktur dieser jahrhundertealten Universi...,Oxford: hier studiert die Elite
7,0,/de/oxford-hier-studiert-die-elite/a-2347044,DEUTSCH LERNEN,0,Top-Thema – Podcast,Immer wieder wird der Universität zu Oxford al...,Oxford: hier studiert die Elite


## Text Corpus QA

In [ ]:
sns.countplot(text_df["y"])

In [ ]:
text_df["text"] = text_df["text"].apply(fetch_dw.oncify_newline_spaces)
text_df["wc"] = text_df["text"].apply(lambda x: len(x.split()))
text_df["pc"] = text_df["text"].apply(lambda x: len(x.split("\n")))

In [ ]:
sns.distplot(text_df[text_df["y"] == 0]["wc"])
sns.distplot(text_df[text_df["y"] == 1]["wc"])

In [ ]:
text_df[text_df["y"] == 1]["pc"].describe(())

In [ ]:
text_df[text_df["y"] == 0]["pc"].describe(())

In [ ]:
sum(text_df[text_df["y"] == 0]["pc"])

In [ ]:
sns.distplot(text_df[text_df["y"] == 0]["pc"], kde=False)

In [ ]:
sns.violinplot(x="y", y ="wc", data=text_df)

In [ ]:
sns.violinplot(x="artikel", y ="wc", data=text_df)

In [ ]:
(text_df["text"]
 .apply(lambda text: max(zip(map(len, text.split()), text.split())))).sort_values().tail()

In [ ]:
tq = text_df[text_df["artikel"] == "Top-Thema – Podcast"].iloc[0]
url = tq["url"]
pq = pages_df[pages_df["url"] == url].iloc[0]
soup = pq["soup"]

In [ ]:
from textstat.textstat import textstat
text_df["FKG"] = text_df["text"].apply(textstat.flesch_kincaid_grade)

sns.distplot(text_df[text_df["y"] == 0]["FKG"], label="0")
sns.distplot(text_df[text_df["y"] == 1]["FKG"], label="1")

plt.legend()
plt.title("Distribution Flesch-Kincaid Grade per Text by Reading Lables")
plt.savefig("dist-fkg.png", dpi=300)

## Topic Anlyasis - LDA

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

no_features = 1000
no_topics = 20
no_top_words = 10

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='german')
tf = tf_vectorizer.fit_transform(text_df["text"])
tf_feature_names = tf_vectorizer.get_feature_names()

# Run LDA
lda = LatentDirichletAllocation(n_topics=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print "Topic %d:" % (topic_idx)
        print " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])

display_topics(lda, tf_feature_names, no_top_words)